<a href="https://colab.research.google.com/github/hsandaver/hsandaver/blob/main/Material_Degradation_Simulator_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install colormath, which is needed to handle LAB color conversions
!pip install colormath

In [ ]:
# Import the necessary libraries for handling images, color conversions, and datasets

# PIL is used for opening and manipulating images
from PIL import Image

# Numpy is used for numerical operations (arrays, pixel data)
import numpy as np

# Pandas is used for loading and handling CSV datasets
import pandas as pd

# Matplotlib is used for plotting heatmaps and displaying images
import matplotlib.pyplot as plt

# Colormath is used for LAB color conversions and color difference calculations
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000

# Requests is used for fetching the image from a URL
import requests
from io import BytesIO

# Math library for applying non-linear fading curves
import math

# Google Colab's "files" module allows us to upload files
from google.colab import files

In [ ]:
# Step 1: Upload the CSV file that contains the color dataset (LAB values)
print("Please upload the CSV file containing the color dataset (e.g., controlled_terms.csv).")

# Prompt the user to upload the CSV file
uploaded_csv = files.upload()

# Check if a CSV file was uploaded
if len(uploaded_csv) == 0:
    print("No CSV file uploaded. Please upload the CSV dataset.")
else:
    # Get the name of the uploaded CSV file
    csv_file_name = list(uploaded_csv.keys())[0]

    # Load the CSV file into a pandas DataFrame
    lab_dataset = pd.read_csv(csv_file_name)
    print(f"CSV file '{csv_file_name}' loaded successfully.")

In [ ]:
# Ask the user for the URL of an image file (JPG, JPEG, or PNG)
print("Please provide the URL of an image file (JPG, JPEG, or PNG) to apply the fading simulation.")
image_url = input("Enter image URL: ")

# Fetch and load the image from the URL
try:
    # Fetch the image from the provided URL
    response = requests.get(image_url)

    # Check if the response is valid
    if response.status_code == 200:
        # Load the image using PIL from the response content
        image = Image.open(BytesIO(response.content))
        print("Image loaded successfully from URL.")

        # Display the image using matplotlib
        plt.imshow(image)
        plt.axis('off')  # Hide axis for a cleaner view
        plt.show()
    else:
        print(f"Error: Unable to fetch the image from the provided URL. HTTP Status Code: {response.status_code}")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
def fade_simulator_paper(lab_pixel, visible_exposure_ratio, uv_exposure_ratio, humidity, temperature):
    """
    Simulates fading for paper based on visible and UV light, humidity, and temperature.

    Parameters:
    - lab_pixel: LAB color values of the pixel (LabColor object)
    - visible_exposure_ratio: Exposure ratio of visible light
    - uv_exposure_ratio: Exposure ratio of UV light (UV is more damaging)
    - humidity: Relative humidity (%) that affects the paper's degradation speed
    - temperature: Temperature (°C) that speeds up or slows down fading

    Returns: Adjusted LAB color values for the faded pixel.
    """
    # Humidity factor: Higher humidity increases fading
    humidity_factor = 1 + (humidity - 50) / 100.0  # 50% RH is neutral

    # Temperature factor: Higher temperature accelerates chemical reactions
    temperature_factor = 1 + (temperature - 20) / 30.0  # 20°C is neutral

    # Paper loses lightness faster under UV and visible light, color shifts more subtly
    lab_pixel.lab_l -= (visible_exposure_ratio * 0.05 + uv_exposure_ratio * 0.15) * humidity_factor * temperature_factor
    lab_pixel.lab_a -= (visible_exposure_ratio * 0.02 + uv_exposure_ratio * 0.05) * humidity_factor * temperature_factor
    lab_pixel.lab_b -= (visible_exposure_ratio * 0.02 + uv_exposure_ratio * 0.05) * humidity_factor * temperature_factor

    return lab_pixel

def fade_simulator_textile(lab_pixel, visible_exposure_ratio, uv_exposure_ratio, humidity, temperature):
    """
    Simulates fading for textiles based on visible and UV light, humidity, and temperature.

    Parameters:
    - lab_pixel: LAB color values of the pixel (LabColor object)
    - visible_exposure_ratio: Exposure ratio of visible light
    - uv_exposure_ratio: Exposure ratio of UV light (UV is more damaging)
    - humidity: Relative humidity (%) that affects the textile's degradation speed
    - temperature: Temperature (°C) that speeds up or slows down fading

    Returns: Adjusted LAB color values for the faded pixel.
    """
    # Humidity factor: Higher humidity affects fibers in textiles
    humidity_factor = 1 + (humidity - 50) / 100.0

    # Temperature factor: Higher temperature accelerates dye fading
    temperature_factor = 1 + (temperature - 20) / 30.0

    # Textiles experience slower lightness loss but larger color shifts
    lab_pixel.lab_l -= (visible_exposure_ratio * 0.03 + uv_exposure_ratio * 0.10) * humidity_factor * temperature_factor
    lab_pixel.lab_a -= (visible_exposure_ratio * 0.04 + uv_exposure_ratio * 0.08) * humidity_factor * temperature_factor
    lab_pixel.lab_b -= (visible_exposure_ratio * 0.04 + uv_exposure_ratio * 0.08) * humidity_factor * temperature_factor

    return lab_pixel

In [ ]:
def non_linear_degradation_curve(exposure_years, max_years=50):
    """
    Simulates faster fading in the early years and slower fading later (non-linear curve).

    Parameters:
    - exposure_years: How many years of light exposure the material has been exposed to.
    - max_years: Maximum years (after which fading slows down).

    Returns: A curve factor that adjusts the speed of fading over time.
    """
    # Use a logarithmic curve to make fading happen faster in the first few years
    curve_factor = math.log1p(exposure_years) / math.log1p(max_years)
    return curve_factor

In [ ]:
# First, install tqdm for progress bar functionality

# Import tqdm for progress bar
from tqdm import tqdm

# =======================
# USER-DEFINED PARAMETERS
# =======================
# The following parameters control the simulation of the fading model.
# You can modify these values to adjust the environmental conditions and material characteristics.

# How many years of light exposure to simulate
exposure_years = 5

# The intensity of visible light in lux (higher values mean more exposure to visible light)
visible_lux = 50  # lux

# The intensity of UV light in lux (higher values mean more exposure to UV light, which is more damaging)
uv_lux = 20  # lux

# The maximum number of exposure hours before critical fading occurs (lux-hours)
max_exposure_hours = 144

# The type of material (choose "paper" or "textile")
material_type = "paper"  # Choose either "paper" or "textile"

# Relative humidity in the environment (percentage)
humidity = 70  # %

# Temperature in the environment (in degrees Celsius)
temperature = 30  # °C

# If the material is a textile, specify the type of textile (choose "natural" or "synthetic")
textile_type = "natural"  # Only used if material_type is "textile"

# If the material is a textile, specify the type of dye (choose "natural" or "synthetic")
dye_type = "natural"  # Only used if material_type is "textile"

# =======================
# FADING SIMULATION FUNCTION
# =======================

def apply_fading_to_image(image, exposure_years=1, visible_lux=50, uv_lux=10, max_exposure_hours=144, material_type="paper", humidity=50, temperature=20, textile_type="natural", dye_type="natural"):
    """
    Applies the fading simulation based on UV/visible light, humidity, temperature, textile type, and dye type.
    If material_type is "paper", textile parameters are ignored.

    Parameters:
    - image: The input image to apply fading to.
    - exposure_years: Number of years the image has been exposed to light.
    - visible_lux: The intensity of visible light (in lux).
    - uv_lux: The intensity of UV light (in lux).
    - max_exposure_hours: The maximum allowable exposure before critical fading.
    - material_type: The type of material ("paper" or "textile").
    - humidity: The relative humidity (%).
    - temperature: The temperature (°C).
    - textile_type: The type of textile ("natural" or "synthetic") (Ignored if material_type is "paper").
    - dye_type: The type of dye ("natural" or "synthetic") (Ignored if material_type is "paper").

    Returns: The faded image.
    """
    # Convert the image to a numpy array with RGB values between 0 and 1
    image_array = np.array(image) / 255.0

    # Initialize an empty array to store the faded LAB values
    lab_image = np.zeros_like(image_array, dtype=float)

    # Calculate exposure in hours (assuming 365 days per year)
    visible_exposure_hours = exposure_years * visible_lux * 365
    uv_exposure_hours = exposure_years * uv_lux * 365

    # Apply non-linear degradation over time (fading is faster in early years)
    time_curve_factor = non_linear_degradation_curve(exposure_years)

    # Inform the user that textile-related parameters are ignored if "paper" is chosen
    if material_type == "paper":
        print("Note: Since 'paper' is selected, textile and dye parameters are ignored.")

    # Loop through each pixel and apply the correct fading model
    for i in tqdm(range(image_array.shape[0]), desc=f"Aging the material for {exposure_years} years..."):
        for j in range(image_array.shape[1]):
            # Convert the pixel from RGB to LAB
            rgb_pixel = sRGBColor(*image_array[i, j])
            lab_pixel = convert_color(rgb_pixel, LabColor)

            # Calculate exposure ratios for visible and UV light
            visible_exposure_ratio = min(visible_exposure_hours / max_exposure_hours, 1.0) * time_curve_factor
            uv_exposure_ratio = min(uv_exposure_hours / max_exposure_hours, 1.0) * time_curve_factor

            # Apply the fading simulator based on the material type
            if material_type == "paper":
                # Apply paper fading logic (textile and dye parameters are not used here)
                lab_pixel = fade_simulator_paper(lab_pixel, visible_exposure_ratio, uv_exposure_ratio, humidity, temperature)
            elif material_type == "textile":
                # Apply textile fading logic (textile and dye parameters are used here)
                lab_pixel = fade_simulator_textile(lab_pixel, visible_exposure_ratio, uv_exposure_ratio, humidity, temperature, textile_type, dye_type)

            # Store the adjusted LAB values in the array
            lab_image[i, j] = [lab_pixel.lab_l, lab_pixel.lab_a, lab_pixel.lab_b]

    # Convert the adjusted LAB image back to RGB and return it
    faded_image = Image.fromarray(np.uint8(lab_image * 255))
    return faded_image

# Example: Apply fading to the image with the defined parameters
faded_image = apply_fading_to_image(image, exposure_years=exposure_years, visible_lux=visible_lux, uv_lux=uv_lux, max_exposure_hours=max_exposure_hours,
                                    material_type=material_type, humidity=humidity, temperature=temperature, textile_type=textile_type, dye_type=dye_type)

# Show the faded image
faded_image.show()

In [ ]:
# Install tqdm for the progress bar
!pip install tqdm

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import rgb2lab
from tqdm import tqdm  # Import tqdm for progress bar

def calculate_cie76_color_difference(lab1, lab2):
    """
    Calculate the color difference (ΔE) between two colors using the CIE76 formula.

    Parameters:
    - lab1: LAB values of the first color (as a NumPy array).
    - lab2: LAB values of the second color (as a NumPy array).

    Returns:
    - delta_e: The color difference (ΔE) between the two colors using CIE76.
    """
    return np.sqrt(np.sum((lab2 - lab1) ** 2))

def color_difference_heatmap(image1, image2, threshold=2):
    """
    Generates a heatmap showing color differences between the original and faded images using CIE76.
    Also calculates the percentage of the image affected by fading (ΔE > threshold).

    Parameters:
    - image1: The original image.
    - image2: The faded image.
    - threshold: The minimum color difference to highlight in the heatmap.

    Returns:
    - A heatmap visualizing the color differences and percentage of the image affected by fading.
    """
    # Convert the images to NumPy arrays with RGB values between 0 and 1
    image1_array = np.array(image1) / 255.0
    image2_array = np.array(image2) / 255.0

    # Convert the RGB images to LAB color space using skimage's rgb2lab
    lab_image1 = rgb2lab(image1_array)
    lab_image2 = rgb2lab(image2_array)

    # Initialize an empty array to store the color differences
    color_diff_map = np.zeros(image1_array.shape[:2])

    # Variables to keep track of affected pixels
    total_pixels = image1_array.shape[0] * image1_array.shape[1]
    affected_pixels = 0  # To count how many pixels exceed the threshold

    # Create a progress bar using tqdm
    print("Analyzing color degradation and comparing original with faded image...")
    for i in tqdm(range(lab_image1.shape[0]), desc="Processing rows", unit="row"):
        for j in range(lab_image1.shape[1]):
            # Calculate the color difference (ΔE) between the two pixels using CIE76
            delta_e = calculate_cie76_color_difference(lab_image1[i, j], lab_image2[i, j])

            # If the color difference exceeds the threshold, record it in the heatmap and count the pixel
            if delta_e > threshold:
                color_diff_map[i, j] = delta_e
                affected_pixels += 1

    # Calculate the percentage of pixels affected by fading
    percentage_affected = (affected_pixels / total_pixels) * 100

    # Plot the color difference heatmap using matplotlib
    plt.imshow(color_diff_map, cmap='hot')
    plt.colorbar()
    plt.title('Color Difference Heatmap (CIE76)')

    # Show the percentage of the image affected by fading
    plt.figtext(0.5, -0.1, f"Percentage of the image affected by fading: {percentage_affected:.2f}%", ha="center", fontsize=12)
    plt.show()

# Example usage: Generate the heatmap comparing the original and faded images
color_difference_heatmap(image, faded_image)

In [ ]:
from ipywidgets import IntSlider, Dropdown, Button, VBox
from IPython.display import display

start_simulation = False

def start_button_clicked(b):
    global start_simulation
    start_simulation = True
    run_simulation()

def run_simulation():
    global start_simulation
    if start_simulation:
        # Assuming 'apply_fading_to_image' and 'image' are defined elsewhere in your code
        faded_image = apply_fading_to_image(image,
                                            exposure_years=exposure_slider.value,
                                            visible_lux=visible_slider.value,
                                            uv_lux=uv_slider.value,
                                            max_exposure_hours=max_exposure_hours,
                                            material_type=material_dropdown.value,
                                            humidity=humidity_slider.value,
                                            temperature=temperature_slider.value,
                                            textile_type=textile_dropdown.value,
                                            dye_type=dye_dropdown.value)
        faded_image.show()
    else:
        print("Please select values and press 'Start Simulation'.")

# Define the sliders and dropdowns with an extended lux range
exposure_slider = IntSlider(min=1, max=100, step=1, value=5, description="Years of Exposure")
visible_slider = IntSlider(min=1, max=10000, step=1, value=500, description="Visible Lux")  # Increased range up to 10,000 lux
uv_slider = IntSlider(min=1, max=1000, step=1, value=100, description="UV Lux")  # Increased range up to 1,000 lux
material_dropdown = Dropdown(options=["paper", "textile"], value="paper", description="Material Type")
humidity_slider = IntSlider(min=0, max=100, step=1, value=50, description="Humidity (%)")
temperature_slider = IntSlider(min=0, max=50, step=1, value=20, description="Temperature (°C)")
textile_dropdown = Dropdown(options=["natural", "synthetic"], value="natural", description="Textile Type (if textile)")
dye_dropdown = Dropdown(options=["natural", "synthetic"], value="natural", description="Dye Type (if textile)")

# Button to start the simulation
start_button = Button(description="Start Simulation")
start_button.on_click(start_button_clicked)

# Display all widgets
widgets = VBox([exposure_slider, visible_slider, uv_slider, material_dropdown,
                humidity_slider, temperature_slider, textile_dropdown, dye_dropdown, start_button])
display(widgets)